In [1]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from bokeh.plotting import figure, show, output_file
import seaborn as sns

In [2]:
train = pd.read_csv('data/train.csv')
train = train.replace('\\N',np.NAN)
n,p = train.shape

In [3]:
def traitement_types(df):
    train_ = df.copy()
    
    for c in train_.columns:
        if 'crd_' in c or c in ['CRD','IMPAYES_DEBUT','age','adulte_foyer']:
            train_[c] = pd.to_numeric(train[c], errors='coerce')
        
    train_.moy_eco_jour = train_.moy_eco_jour.str.replace(',', '.', regex=False).astype(float)
    train_.RAV_UC = train_.RAV_UC.str.replace(',', '.', regex=False).astype(float)
    
    train_['Date'] = pd.to_datetime(train_.year.astype(str) + '-' + train_.month.astype(str), format='%Y-%m')
    train_ = train_.drop(columns=['year','month'])
    
    return train_


In [4]:
def trancheAge(x):
    age = x.age
    if age < 25:
        return '<25ans'

    elif age >= 25 and age <= 34:
        return  '25-34ans'

    elif age >= 35 and age <= 44:
        return  '35-44ans'

    elif age >= 45 and age <= 54:
        return  '45-54ans'

    elif age >= 55 and age <= 64:
        return  '55-64ans'

    elif age >= 65 and age <= 74:
        return  '65-74ans'

    elif age >= 75:
        return '>75ans'
    
    
def traitement_na(df):
    train_ = df.copy()
    
    train_ = train_.replace('Non Renseigne', np.NAN)
    for CRD in train.columns.tolist():
        if 'crd_' in CRD:
            train_[CRD] = train_[CRD].fillna(0)
    
    for p in train_.PROF.unique():
        med = train_.loc[train_.PROF == p].age.median()
        train_.loc[train_.PROF == p, 'age'] = train_.loc[train_.PROF == p, 'age'].fillna(med)
    
    train_.tranche_age = train_.apply(lambda x: trancheAge(x), axis=1)
    
    train_ = train_.drop(columns=['IMPAYES_DEBUT'])
    
    train_.gain_mediation = train_.gain_mediation.fillna(0).astype(int)
    
    train_.cat_impayes = train_.cat_impayes.fillna('Inconnu')
    
    train_ = train_.dropna()
    
    return train_

In [5]:
train = traitement_types(train)
train = traitement_na(train)

cols_categorielles = list(train.dtypes[train.dtypes == 'object'].to_dict().keys())

for cc in cols_categorielles:
    cat_dtype = pd.api.types.CategoricalDtype(categories=train[cc].unique().tolist(), ordered=False)
    train[cc] = train[cc].astype(cat_dtype)

train.shape


# I - Kmeans

In [7]:
from sklearn.cluster import KMeans

In [9]:
non_cat = train.drop(columns=cols_categorielles+['Date'])

(25, 25)